In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
from numba import njit, prange

In [ ]:
@njit
def laplacian2D(Z,dx,dy):

    center  = Z[1:-1, 1:-1]
    
    w   = Z[0:-2, 1:-1]
    e   = Z[2:  , 1:-1]   
    n   = Z[1:-1, 2:  ]
    s   = Z[1:-1, 0:-2]
    
    ne  = Z[2:  , 2:  ]
    nw  = Z[0:-2, 2:  ] 
    se  = Z[2:, 0:-2]
    sw  = Z[0:-2, 0:-2]
    
    dif2 = (w+e+n+s)/6.0 + (ne+nw+se+sw)/12.0 
    dif2 = dif2 - center
    
    dif2 = dif2/dx**2
    return dif2

@njit    
def PBC(Z):
    Z[0,:]  = Z[-2,:]
    Z[-1,:] = Z[1,:]
    Z[:,0]  = Z[:,-2]
    Z[:,-1] = Z[:,1]
    Z[-2,:] = Z[0,:]
    Z[0,:]  = Z[-1,:]
    return Z

@njit
def NMC(Z):
    Z[0,:] = Z[1,:]
    Z[-1,:] = Z[-2,:]
    Z[:,0] = Z[:,1]
    Z[:,-1] = Z[:,-2]
    return Z

@njit
def HS(c):
    return c*(3*c**2 - 9*c + 8)/((1-c)**3)

In [ ]:
@njit(parallel=True)
def solve_dynamics(s, c, dx, dy, dt, n, T, R, alpha, r):
    for i in prange(n):
        aux = c + alpha * s**2 
        nabla2_aux = laplacian2D(aux,dx,dy)
        nabla2_s = laplacian2D(s,dx,dy)
 
        cc = c[1:-1,1:-1]
        sc = s[1:-1,1:-1]
        
        s[1:-1,1:-1] = sc + dt*(nabla2_s - r*sc - sc**3 - sc*cc)
        c[1:-1,1:-1] = cc + dt * R * nabla2_aux
        
        for x in (s,c):
            PBC(x)

In [ ]:
def save_fields(s, c, step, nb):
    LC_file = "matrix" + "_s_" + str(step) + "_nb_" + str(nb) + ".txt"
    np.savetxt(LC_file, s, delimiter="\t")
    
    NPs_file = "matrix" + "_c_" + str(step) + "_nb_" + str(nb) + ".txt"
    np.savetxt(NPs_file, c, delimiter="\t")

In [ ]:
N=128

dx = 1.0
dy = dx
T = 1
dt = 0.01
n = int(T/dt)
print(n)

R = 0.10
alpha = 0.05
r = -1.0

nb = 21 #numero de bloques

#plt.imshow(c, cmap=plt.cm.gray)
#plt.show()
        
t0 = time.time()
for block in range(1, nb):
    c = np.random.uniform(0.009, 0.011,  (N, N))
    s = np.random.uniform(-0.001, 0.001, (N, N))
    for x in (s,c):
        PBC(x)
    save_fields(s, c, 0, block)
    for i in range(1,101):
        solve_dynamics(s, c, dx, dy, dt, n, T, R, alpha, r)
        save_fields(s, c, i, block)
tf = time.time()

plt.imshow(c, cmap=plt.cm.gray)
plt.show()

Deltat = tf-t0
print("Tiempo real (s) (min) (hrs): ", Deltat, Deltat/60, Deltat/3600)